## US Sales Online

In [1]:
import pandas as pd
import sqlite3

# Read the CSV file into a DataFrame
osus = pd.read_csv('/Users/nikhilsharma/Desktop/Data/Datasets/Old/Online Sales in US.csv')

# Display the first few rows of the DataFrame
print(osus.columns)

# Establish a connection to the SQLite database
cnn = sqlite3.connect('online_sales_us.db')

# Create a cursor object
cursor = cnn.cursor()

# Drop the 'marc' table if it already exists
cursor.execute('DROP TABLE IF EXISTS osus')

# Write the DataFrame to an SQLite table
osus.to_sql('osus', cnn)

# Perform SQL queries using sqlite3
cursor.execute('SELECT * FROM osus LIMIT 5')
results = cursor.fetchall()
for row in results:
    print(row)


## Analysis using SQL

#--Average Quantity & Purchase Value by Customer

cursor.execute('''SELECT cust_id, "First Name" || ' ' || "Middle Initial" || ' ' || "Last Name" AS name,
CAST(AVG(qty_ordered) AS INT64) AS avg_quantity, ROUND(AVG(total), 0) AS avg_purchase_value
FROM osus
GROUP BY cust_id, name
ORDER BY AVG(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)

#--Total Quantity & Purchase Value by Customer

cursor.execute('''SELECT cust_id, "First_Name" || ' ' || "Middle_Initial" || ' ' || "Last_Name" AS "name", CAST(SUM(qty_ordered) AS INT64) AS "total_quantity", ROUND(SUM(total), 0) "total_purchase_value"
FROM osus 
GROUP BY cust_id, name
ORDER BY SUM(total) DESC
LIMIT 100''')
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)

#--Most sold product & product with highest total sale amount

cursor.execute('''SELECT sku,ROUND(SUM(total),0) AS "total_sum", COUNT(sku) AS "cnt_sku"
FROM osus
GROUP BY sku
ORDER BY COUNT(sku) DESC
LIMIT 10''')
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)

#--No. and % of orders that were not refunded or canceled

cursor.execute('''SELECT COUNT(*) AS total_orders, (SELECT COUNT(status)
FROM osus
WHERE status IN ('order_refunded', 'canceled', 'refund')) AS refund_or_canc, COUNT(*) - (SELECT COUNT(status)
FROM osus
WHERE status IN ('order_refunded', 'canceled', 'refund')) AS orders,
ROUND(((SELECT COUNT(status)
FROM osus
WHERE status IN ('order_refunded', 'canceled', 'refund'))/COUNT(*) * 100),2)  AS orders_ref_canc_perc

FROM osus
LIMIT 10''')
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)

#-- Sale by Place

cursor.execute('''SELECT "Place Name", CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY "Place Name"
ORDER BY SUM(total) DESC
LIMIT 10''')
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#-- Sale by County

cursor.execute('''SELECT County, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY County
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)

#-- Sale by City

cursor.execute('''SELECT City, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY City
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)

#-- Sale by State

cursor.execute('''SELECT State, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY State
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#-- Sale by ZIP

cursor.execute('''SELECT Zip, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY Zip
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#-- Sale by Region

cursor.execute('''SELECT region, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY region
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--No. of Orders with & without discount

cursor.execute('''SELECT 
(SELECT COUNT(*) AS non_discount_orders FROM osus WHERE discount_amount <= 0) AS no_disc_orders, 
(SELECT COUNT(*) AS discount_orders FROM osus WHERE discount_amount <> 0) AS disc_orders
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Total Revenue based on Customer Since (how long the customer has been with the company)

# By Total Revenue
cursor.execute('''SELECT strftime('%Y', "Customer Since") as cust_since_year, SUM(total) AS total_revenue
FROM osus
GROUP BY cust_since_year
ORDER BY total_revenue DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)




# By Customer Since Year
cursor.execute('''SELECT strftime('%Y', "Customer Since") as cust_since_year, CAST(SUM(total) AS INTEGER) AS total_revenue
FROM osus
GROUP BY cust_since_year
ORDER BY cust_since_year DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Total Revenue by Payment Method

cursor.execute('''SELECT payment_method, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY payment_method
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Average Revenue by Payment Method

cursor.execute('''SELECT payment_method, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue
FROM osus
GROUP BY payment_method
ORDER BY AVG(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Total by Order Status

cursor.execute('''SELECT status, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY status
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Average by Order Status

cursor.execute('''SELECT status, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY status
ORDER BY AVG(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Total Revenue per Month

cursor.execute('''SELECT month, CAST(SUM(total) AS INT64) AS total_revenue 
FROM osus
GROUP BY month
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Total Revenue per Age

cursor.execute('''SELECT age, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY age
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Average Revenue per Age

cursor.execute('''SELECT age, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY age
ORDER BY AVG(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Total Revenue per Gender

cursor.execute('''SELECT gender, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY gender
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Average Revenue per Gender

cursor.execute('''SELECT gender, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY gender
ORDER BY AVG(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Total by Category

cursor.execute('''SELECT Category, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY Category
ORDER BY SUM(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)


#--Average by Category

cursor.execute('''SELECT Category, CAST(SUM(total) AS INT64) AS total_revenue, CAST(AVG(total) AS INT64) AS avg_revenue 
FROM osus
GROUP BY Category
ORDER BY AVG(total) DESC
LIMIT 100''')

# Fetch the results and create a DataFrame
results = cursor.fetchall()
column_names = [description[0] for description in cursor.description]

# Create a DataFrame from the results with column names
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
print(df)

# Close the database connection
cnn.close()

Index(['order_id', 'order_date', 'status', 'item_id', 'sku', 'qty_ordered',
       'price', 'value', 'discount_amount', 'total', 'category',
       'payment_method', 'bi_st', 'cust_id', 'year', 'month', 'ref_num',
       'Name Prefix', 'First Name', 'Middle Initial', 'Last Name', 'Gender',
       'age', 'full_name', 'E Mail', 'Customer Since', 'SSN', 'Phone No. ',
       'Place Name', 'County', 'City', 'State', 'Zip', 'Region', 'User Name',
       'Discount_Percent'],
      dtype='object')
(0, 100354678, '10/1/20', 'received', 574772, 'oasis_Oasis-064-36', 21, 89.9, 1798.0, 0.0, 1798.0, "Men's Fashion", 'cod', 'Valid', 60124, 2020, 'Oct-20', 987867, 'Drs.', 'Jani', 'W', 'Titus', 'F', 43, 'Titus, Jani', 'jani.titus@gmail.com', '8/22/06', '627-31-5251', '405-959-1129', 'Vinson', 'Harmon', 'Vinson', 'OK', 73571, 'South', 'jwtitus', 0.0)
(1, 100354678, '10/1/20', 'received', 574774, 'Fantastic_FT-48', 11, 19.0, 190.0, 0.0, 190.0, "Men's Fashion", 'cod', 'Valid', 60124, 2020, 'Oct-20', 9878